In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

In [2]:
# dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz",
#                                   "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
#                                   True,
#                                   cache_dir= '.',
#                                  cache_subdir='.')

In [3]:
path = os.path.dirname("aclImdb_v1.tar.gz")
print(path)

In [4]:
dataset_dir  = os.path.join(path, "aclImdb")
print(dataset_dir)

aclImdb


In [5]:
train_dir = os.path.join(dataset_dir, "train")

In [7]:
print(train_dir)

aclImdb\train


In [8]:
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [9]:
# remove_dir = os.path.join( train_dir, "unsup") this step is already done

In [10]:
# shutil.rmtree(remove_dir)  this step is already done

In [11]:
batch_size = 1024

In [12]:
seed = 64

In [13]:
train_ds = tf.keras.preprocessing.text_dataset_from_directory(train_dir, batch_size = batch_size, validation_split=0.2 , subset="training", seed = seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [14]:
val_ds = tf.keras.preprocessing.text_dataset_from_directory(train_dir, batch_size = batch_size, validation_split = 0.2 , subset="validation", seed= seed)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [15]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(5):
        print(label_batch[i].numpy(), text_batch[i].numpy())

0 b'This film\'s trailer interested me enough to warrant renting the DVD. However, the resulting movie is absolutely dire! Admittedly, this is not the worst film ever made, or the worst film this year, but it came damn close!<br /><br />The main issue is the film not knowing what it wants to be: comedy, adult drama, thriller, teen-porn? The story is interesting, as it deals with the pitfalls of mail-order brides, but the film is a mess. What starts out as a mildly interesting "comedy" (a word I use in the loosest possible terms), then goes totally in reverse, and degenerates into a very dark and distasteful misogynistic thriller. Nicole Kidman should know better, and Ben Chaplin is wasted! As are Matthieu Kassovitz and Vincent Cassel, whom I can only presume did this for the money.<br /><br />This is a bad film in pretty much every single aspect. It\'s not funny, it\'s almost so sexist that you could almost forgive Benny Hill for everything he did, and the dramatic elements are just do

In [16]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [17]:
embedding_layer = tf.keras.layers.Embedding(1000, 5)

In [22]:
result = embedding_layer(tf.constant([3, 7, 9, 10, 11, 200]))

In [23]:
result

<tf.Tensor: shape=(6, 5), dtype=float32, numpy=
array([[ 0.02511052,  0.01828988, -0.01169553, -0.00181482,  0.02978926],
       [-0.00285812,  0.00912962,  0.04531452, -0.0196569 , -0.02837334],
       [ 0.02840414,  0.01021618, -0.04309234,  0.00395962, -0.03067114],
       [ 0.00883833,  0.03317884, -0.03403064,  0.02822263,  0.01268062],
       [-0.0349094 ,  0.03444279,  0.03563032,  0.04942328,  0.04480261],
       [-0.02943516, -0.01693821,  0.00580652, -0.02758094,  0.0145897 ]],
      dtype=float32)>

In [24]:
len(embedding_layer.get_weights()[0])

1000

In [25]:
result = embedding_layer(tf.constant([[0,1,2,3],[3,4,5,6]]))

In [26]:
result

<tf.Tensor: shape=(2, 4, 5), dtype=float32, numpy=
array([[[ 0.0355163 ,  0.04144997, -0.03554094,  0.02133859,
          0.02183565],
        [-0.01927599, -0.00717426,  0.04862859, -0.02970214,
          0.04600758],
        [ 0.00317008, -0.00040751,  0.01963696,  0.02729354,
          0.00213166],
        [ 0.02511052,  0.01828988, -0.01169553, -0.00181482,
          0.02978926]],

       [[ 0.02511052,  0.01828988, -0.01169553, -0.00181482,
          0.02978926],
        [ 0.0402602 ,  0.04003313,  0.02967794, -0.04804343,
          0.0483623 ],
        [ 0.04866591, -0.01627334, -0.01257658,  0.01128574,
         -0.0135993 ],
        [-0.04244554, -0.03557456, -0.03991872, -0.02145844,
         -0.00908531]]], dtype=float32)>

In [27]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br/', ' ')
    final_data = tf.strings.regex_replace(stripped_html, re.escape(string.punctuation), '')
    print(final_data)
    return final_data

In [28]:
vocab_size = 10000
sequence_length = 100

In [29]:
vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
    standardize = custom_standardization,
    max_tokens  = vocab_size,
    output_mode = 'int', 
    output_sequence_length = sequence_length
)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)

In [30]:
adapt = vectorize_layer.adapt(text_ds)

Tensor("StaticRegexReplace_1:0", shape=(None, 1), dtype=string)


In [31]:
vectorize_layer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'a',
 'and',
 'of',
 'to',
 'is',
 'in',
 'i',
 'this',
 'that',
 'it',
 '/><br',
 'was',
 'as',
 'for',
 'with',
 'but',
 'on',
 'movie',
 'his',
 'are',
 'not',
 'film',
 'you',
 'have',
 'he',
 'be',
 'at',
 'one',
 'by',
 'an',
 'they',
 'from',
 'all',
 'who',
 'like',
 'so',
 'just',
 'or',
 'has',
 'about',
 'her',
 "it's",
 'some',
 'if',
 'out',
 'what',
 'very',
 'when',
 'more',
 'there',
 'she',
 'would',
 'good',
 'even',
 'my',
 'only',
 'no',
 'their',
 'had',
 'really',
 'which',
 'can',
 'up',
 'were',
 'see',
 'than',
 '-',
 'we',
 'been',
 'get',
 'will',
 'into',
 'story',
 'because',
 'much',
 'most',
 'how',
 'other',
 'also',
 'its',
 "don't",
 'time',
 'do',
 'first',
 'great',
 'people',
 'me',
 'could',
 'make',
 'any',
 '/>the',
 'after',
 'made',
 'then',
 'bad',
 'think',
 'him',
 'many',
 'never',
 'being',
 'two',
 'too',
 'where',
 'little',
 'well',
 '<br',
 'way',
 'watch',
 'your',
 'it.',
 'did',
 'does',
 'them',
 'know',
 'b

In [32]:
model = tf.keras.models.Sequential()
# Start by creating an explicit input layer. It needs to have a shape of
# (1,) (because we need to guarantee that there is exactly one string
# input per batch), and the dtype needs to be 'string'.
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)

Tensor("text_vectorization/StaticRegexReplace_1:0", shape=(None, 1), dtype=string)


In [33]:
input_data = [["the movie was great"], ["hero did a great work"]]
pred = model.predict(input_data)

Tensor("sequential/text_vectorization/StaticRegexReplace_1:0", shape=(None, 1), dtype=string)


In [34]:
pred.shape

(2, 100)

In [35]:
pred[0].shape

(100,)

In [36]:
pred

array([[  2,  20,  14,  87,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [909, 113,   3,  87, 210,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [37]:
embedding_dim=16

model = tf.keras.models.Sequential([
  vectorize_layer,
  tf.keras.layers.Embedding(vocab_size, embedding_dim, name="embedding"),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(1)
])

In [38]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/embedding")

In [39]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [40]:
model.fit(
    train_ds,
    validation_data=val_ds, 
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
Tensor("text_vectorization/StaticRegexReplace_3:0", shape=(None, 1), dtype=string)
Tensor("sequential_1/text_vectorization/StaticRegexReplace_1:0", shape=(None, 1), dtype=string)
Tensor("sequential_1/text_vectorization/StaticRegexReplace_1:0", shape=(None, 1), dtype=string)
 1/20 [>.............................] - ETA: 0s - loss: 0.6933 - accuracy: 0.5156WARNING:tensorflow:From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
20/20 [==============================] - 21s 1s/step - loss: 0.6916 - accuracy: 0.5009 - val_loss: 0.6888 - val_accuracy: 0.4964
Epoch 2/15
20/20 [==============================] - 6s 287ms/step - loss: 0.6851 - accuracy: 0.5009 - val_loss: 0.6814 - val_accuracy: 0.4964
Epoch 3/15
20/20 [==============================] - 6s 288ms/step - loss: 0.675

In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [42]:
%load_ext tensorboard

In [43]:
%reload_ext tensorboard
%tensorboard --logdir logs/embedding

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 12872.

# WordEmbedding Replacing GlobalAveragePooling1D With Flatten Layer

In [44]:
embedding_dim=16

model_faltten = tf.keras.models.Sequential([
  vectorize_layer,
  tf.keras.layers.Embedding(vocab_size, embedding_dim, name="embedding"),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(1)
])

In [45]:
model_faltten.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model_faltten.fit(
    train_ds,
    validation_data=val_ds, 
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
Tensor("text_vectorization/StaticRegexReplace_5:0", shape=(None, 1), dtype=string)
Tensor("sequential_2/text_vectorization/StaticRegexReplace_1:0", shape=(None, 1), dtype=string)
Tensor("sequential_2/text_vectorization/StaticRegexReplace_1:0", shape=(None, 1), dtype=string)
20/20 [==============================] - 8s 418ms/step - loss: 0.6924 - accuracy: 0.5009 - val_loss: 0.6902 - val_accuracy: 0.4964
Epoch 2/15
20/20 [==============================] - 6s 285ms/step - loss: 0.6754 - accuracy: 0.5012 - val_loss: 0.6692 - val_accuracy: 0.4992
Epoch 3/15
20/20 [==============================] - 6s 283ms/step - loss: 0.6114 - accuracy: 0.5709 - val_loss: 0.5855 - val_accuracy: 0.6316
Epoch 4/15
20/20 [==============================] - 6s 288ms/step - loss: 0.4814 - accuracy: 0.7598 - val_loss: 0.4841 - val_accuracy: 0.7624
Epoch 5/15
20/20 [==============================] - 6s 279ms/step - loss: 0.3611 - accuracy: 0.8443 - val_loss: 0.4333 - val_accuracy: 0.7936
Epoch 6/15
20/2

In [46]:
model_faltten.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/new123

In [ ]:
weights = model.get_layer('embedding').get_weights()

In [ ]:
len(weights[0])

In [ ]:
weights[0][9997]

In [ ]:
weights

In [ ]:
model.predict([["isha speaks bad word"],["neha  is silent girl and smart "]])

In [ ]:
model_faltten.predict([["isha speaks bad word"],["neha  is silent girl and smart "]])

In [ ]:
embedding = model.get_layer('embedding')

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')
for index, word in enumerate(vocab):
    if  index == 0: continue # skip 0, it's padding.
    vec = weights[index] 
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")
out_v.close()
out_m.close()